In [1]:
from flask import Flask, render_template, redirect, jsonify

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import json
from datetime import datetime
from pathlib import Path
import pandas as pd

import time
import pickle

  
# Database Setup
connection_string = "postgres:Golfer7!@localhost:5432/afl_statistics_DB"
engine = create_engine(f'postgresql://{connection_string}')

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
afl_table = Base.classes.afl_team_performance

In [2]:
# Sklearn Packages
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Sklearn Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error, precision_score, confusion_matrix, accuracy_score

In [105]:
df = pd.read_sql_query('select * from "afl_team_performance_last_5_games_diff_percent"',con=engine)

In [106]:
# Visualizes all the columns
pd.set_option('display.max_columns', None)

In [107]:
df.head()

,team,team2,gameid,date,year,round,venue,starttime,home_away,team_score,rainfall,team_points,opposing_team_score,win_loss_margin,win_loss_margin_percent,win_loss,disposals,kicks,marks,handballs,goals,behinds,hitouts,tackles,rebounds,inside50s,clearances,clangers,frees,frees_against,contested_possessions,uncontested_possessions,contested_marks,marks_inside50,one_percenters,bounces,goal_assists
0,Fremantle,Geelong,2012EF01,2012-09-08,2012,EF,M.C.G.,19:45:00,0,-6.500645,0.36,-14.280614,-7.015041,-0.000439,99.924300,1,0.291371,4.556246,3.614276,-6.310085,-7.689843,14.625229,17.466144,-10.088272,-1.766472,-2.741980,0.276970,-1.952066,-4.305190,-10.045444,0.567356,-1.322849,-9.349155,-8.820287,1.093972,-10.704728,-7.212777
1,Geelong,Fremantle,2012EF01,2012-09-08,2012,EF,M.C.G.,19:45:00,1,6.500645,1.32,14.280614,7.015041,0.000439,100.075815,0,-0.291371,-4.556246,-3.614276,6.310085,7.689843,-14.625229,-17.466144,10.088272,1.766472,2.741980,-0.276970,1.952066,4.305190,10.045444,-0.567356,1.322849,9.349155,8.820287,-1.093972,10.704728,7.212777
2,West Coast,North Melbourne,2012EF02,2012-09-09,2012,EF,Subiaco,13:15:00,1,-5.717059,2.12,-14.280614,-4.358695,-0.001292,100.418410,1,-5.015004,-4.844844,-12.595148,-5.236428,-7.243752,7.889546,24.321585,-6.302700,-4.045658,-0.404817,7.691251,-3.921184,9.706382,-2.777135,2.199333,-9.850267,11.994003,11.105969,-4.608757,-10.379678,-6.020470
3,North Melbourne,West Coast,2012EF02,2012-09-09,2012,EF,Subiaco,13:15:00,0,5.717059,1.36,14.280614,4.358695,0.001292,99.585062,0,5.015004,4.844844,12.595148,5.236428,7.243752,-7.889546,-24.321585,6.302700,4.045658,0.404817,-7.691251,3.921184,-9.706382,2.777135,-2.199333,9.850267,-11.994003,-11.105969,4.608757,10.379678,6.020470
4,Hawthorn,Sydney,2012GF01,2012-09-29,2012,GF,M.C.G.,14:30:00,1,4.788508,4.28,24.992190,0.963797,0.001829,99.853157,0,-1.061557,0.525549,1.152295,-3.044685,3.844921,25.702047,-5.763689,-7.185091,-16.424146,10.306482,4.491195,0.626239,-6.449879,9.601807,-0.590920,-0.714908,-29.985007,-1.753617,1.320023,-7.996002,3.569835


In [108]:
index = df.index
number_of_rows = len(index)
print(number_of_rows)

3994


In [109]:
# Drop the null rows
df = df.dropna()
df.head()

,team,team2,gameid,date,year,round,venue,starttime,home_away,team_score,rainfall,team_points,opposing_team_score,win_loss_margin,win_loss_margin_percent,win_loss,disposals,kicks,marks,handballs,goals,behinds,hitouts,tackles,rebounds,inside50s,clearances,clangers,frees,frees_against,contested_possessions,uncontested_possessions,contested_marks,marks_inside50,one_percenters,bounces,goal_assists
0,Fremantle,Geelong,2012EF01,2012-09-08,2012,EF,M.C.G.,19:45:00,0,-6.500645,0.36,-14.280614,-7.015041,-0.000439,99.924300,1,0.291371,4.556246,3.614276,-6.310085,-7.689843,14.625229,17.466144,-10.088272,-1.766472,-2.741980,0.276970,-1.952066,-4.305190,-10.045444,0.567356,-1.322849,-9.349155,-8.820287,1.093972,-10.704728,-7.212777
1,Geelong,Fremantle,2012EF01,2012-09-08,2012,EF,M.C.G.,19:45:00,1,6.500645,1.32,14.280614,7.015041,0.000439,100.075815,0,-0.291371,-4.556246,-3.614276,6.310085,7.689843,-14.625229,-17.466144,10.088272,1.766472,2.741980,-0.276970,1.952066,4.305190,10.045444,-0.567356,1.322849,9.349155,8.820287,-1.093972,10.704728,7.212777
2,West Coast,North Melbourne,2012EF02,2012-09-09,2012,EF,Subiaco,13:15:00,1,-5.717059,2.12,-14.280614,-4.358695,-0.001292,100.418410,1,-5.015004,-4.844844,-12.595148,-5.236428,-7.243752,7.889546,24.321585,-6.302700,-4.045658,-0.404817,7.691251,-3.921184,9.706382,-2.777135,2.199333,-9.850267,11.994003,11.105969,-4.608757,-10.379678,-6.020470
3,North Melbourne,West Coast,2012EF02,2012-09-09,2012,EF,Subiaco,13:15:00,0,5.717059,1.36,14.280614,4.358695,0.001292,99.585062,0,5.015004,4.844844,12.595148,5.236428,7.243752,-7.889546,-24.321585,6.302700,4.045658,0.404817,-7.691251,3.921184,-9.706382,2.777135,-2.199333,9.850267,-11.994003,-11.105969,4.608757,10.379678,6.020470
4,Hawthorn,Sydney,2012GF01,2012-09-29,2012,GF,M.C.G.,14:30:00,1,4.788508,4.28,24.992190,0.963797,0.001829,99.853157,0,-1.061557,0.525549,1.152295,-3.044685,3.844921,25.702047,-5.763689,-7.185091,-16.424146,10.306482,4.491195,0.626239,-6.449879,9.601807,-0.590920,-0.714908,-29.985007,-1.753617,1.320023,-7.996002,3.569835


In [110]:
# Set features. This will also be used as your x values.
features = df.drop(columns=['date','venue','round','starttime', 'team_score', 'opposing_team_score', 'win_loss_margin', 'win_loss_margin_percent','goals','behinds'])
target = df.win_loss.values.reshape(-1,1)


In [111]:
print(features.shape, target.shape)

(3994, 27) (3994, 1)


In [112]:
# check the correlations of the columns to win_loss
features.corr().win_loss

year                      -0.010392
home_away                  0.119187
rainfall                   0.002235
team_points                0.296272
win_loss                   1.000000
disposals                  0.167654
kicks                      0.194757
marks                      0.044466
handballs                  0.067540
hitouts                    0.042413
tackles                    0.076028
rebounds                  -0.164692
inside50s                  0.306674
clearances                 0.132736
clangers                  -0.034808
frees                     -0.009848
frees_against              0.008564
contested_possessions      0.220625
uncontested_possessions    0.082129
contested_marks            0.137089
marks_inside50             0.227590
one_percenters             0.099416
bounces                   -0.021507
goal_assists               0.243224
Name: win_loss, dtype: float64

In [113]:
features = features.drop(columns=['rainfall','rebounds', 'clangers','frees_against', 'hitouts', 'frees','one_percenters', 'marks', 'handballs', 'tackles', 'uncontested_possessions','bounces'])

In [114]:
# check the correlations of the columns to win_loss
features.corr().win_loss

year                    -0.010392
home_away                0.119187
team_points              0.296272
win_loss                 1.000000
disposals                0.167654
kicks                    0.194757
inside50s                0.306674
clearances               0.132736
contested_possessions    0.220625
contested_marks          0.137089
marks_inside50           0.227590
goal_assists             0.243224
Name: win_loss, dtype: float64

In [115]:
#X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1, test_size=.40)

In [116]:
X_train = features[features['year'] < 2018]
X_test = features[(features['year'] > 2017)]
#X_test = features[(features['year'] > 2017) & (features['year'] < 2022)]
# Copy data for the second model - H20
X2_train = X_train
X2_test = X_test
X3_train = X_train
X3_test = X_test

#y_train = X_train.win_loss.values.reshape(-1,1)
#y_test = X_test.win_loss.values.reshape(-1,1)
y_train = X_train.win_loss.values
y_test = X_test.win_loss.values
y2_train = y_train
y2_test = y_test
y3_train = y_train
y3_test = y_test
                  
X_train = X_train.drop(columns=['win_loss','team','team2','gameid'])     
X_test = X_test.drop(columns=['win_loss','team','team2','gameid'])    

In [117]:
# Checking if train test split ran correclty
for dataset in [y_train, y_test]:
    print(round(len(dataset)/len(target), 2))

0.6
0.4


In [118]:
y_test.shape

(1584,)

In [119]:
X_train.head()

,year,home_away,team_points,disposals,kicks,inside50s,clearances,contested_possessions,contested_marks,marks_inside50,goal_assists
0,2012,0,-14.280614,0.291371,4.556246,-2.741980,0.276970,0.567356,-9.349155,-8.820287,-7.212777
1,2012,1,14.280614,-0.291371,-4.556246,2.741980,-0.276970,-0.567356,9.349155,8.820287,7.212777
2,2012,1,-14.280614,-5.015004,-4.844844,-0.404817,7.691251,2.199333,11.994003,11.105969,-6.020470
3,2012,0,14.280614,5.015004,4.844844,0.404817,-7.691251,-2.199333,-11.994003,-11.105969,6.020470
4,2012,1,24.992190,-1.061557,0.525549,10.306482,4.491195,-0.590920,-29.985007,-1.753617,3.569835


In [120]:
X_test.head()

,year,home_away,team_points,disposals,kicks,inside50s,clearances,contested_possessions,contested_marks,marks_inside50,goal_assists
2410,2018,0,-14.280614,1.191077,-0.466407,-1.946730,3.866869,0.652720,-18.454441,-7.426450,1.638673
2411,2018,1,14.280614,-1.191077,0.466407,1.946730,-3.866869,-0.652720,18.454441,7.426450,-1.638673
2412,2018,1,19.990005,0.963159,-0.796981,-4.191198,-1.369675,2.646147,10.165184,-5.879471,-9.750152
2413,2018,0,-19.990005,-0.963159,0.796981,4.191198,1.369675,-2.646147,-10.165184,5.879471,9.750152
2414,2018,0,0.000000,4.364065,-1.475157,1.811430,0.817327,2.053849,-10.933229,6.419074,-4.542873


In [121]:
# Evaluation function

def evaluation(y_true, y_pred):
    
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Accuracy: ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('Recall: ' + str(metrics.recall_score(y_test, y_pred)))
    print('F1 Score: ' + str(metrics.f1_score(y_test, y_pred)))
    print('Precision: ' + str(metrics.precision_score(y_test, y_pred)))
    
# Print Confusion Matrix
    print('\nConfusion Matrix:')
    print(' TN,  FP, FN, TP')
    print(confusion_matrix(y_true, y_pred).ravel())
    
# Function Prints best parameters for GridSearchCV
def print_results(results):
    print('Best Parameters: {}\n'.format(results.best_params_)) 

In [122]:
# RandomForest Model
forest = RandomForestClassifier(n_estimators=100, criterion='gini')
#forest = RandomForestClassifier(n_estimators=100, criterion='gini')
# Fitting Model to the train set
forest.fit(X_train, y_train)
# Predicting on the test set
y_pred = forest.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.6079545454545454
Recall: 0.5958815958815958
F1 Score: 0.5985778926955397
Precision: 0.6012987012987013

Confusion Matrix:
 TN,  FP, FN, TP
[500 307 314 463]


In [123]:
# Logistic Regression Model
lr = LogisticRegression(C=100, max_iter=400, class_weight='balanced')

# Fitting Model to the train set
lr.fit(X_train, y_train)

# Predicting on the test set
y_pred = lr.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.6231060606060606
Recall: 0.6241956241956242
F1 Score: 0.6190172303765157
Precision: 0.6139240506329114

Confusion Matrix:
 TN,  FP, FN, TP
[502 305 292 485]


In [124]:
y_pred

array([0, 1, 1, ..., 1, 0, 1], dtype=int64)

In [125]:
# Linear Regression Model - setup model for reuse by flask application
regressor = LinearRegression()

In [126]:
regressor.fit(X_train,y_train)

LinearRegression()

In [127]:
# Saving model to current directory
# Pickle serializes objects so they can be saved to a file, and loaded in a program again later on.
pickle.dump(regressor, open('Resources/model.pkl','wb'))

In [128]:
#Loading model to compare the results
model = pickle.load(open('Resources/model.pkl','rb'))

In [129]:
#print(model.predict(X_test))
lr_pred=model.predict(X_test)

In [130]:

df_lr=pd.DataFrame(lr_pred, columns=['model_pr']) 

In [131]:

df_lr.loc[df_lr['model_pr']<0.5,'win_loss'] = 0
df_lr.loc[df_lr['model_pr']>0.4999,'win_loss'] = 1

In [132]:
lr_win_loss = df_lr['win_loss'].to_list()

In [133]:
evaluation(y_test, lr_win_loss)

Evaluation Metrics:
Accuracy: 0.6237373737373737
Recall: 0.6087516087516087
F1 Score: 0.6134889753566797
Precision: 0.6183006535947713

Confusion Matrix:
 TN,  FP, FN, TP
[515 292 304 473]


In [134]:
lr_win_loss

[0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0

In [135]:
# Add the round 1 predictions to the test dataframe
X3_test['Model_win_loss'] = df_lr['win_loss'].values

<ipython-input-135-88dddcb3872e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X3_test['Model_win_loss'] = df_lr['win_loss'].values


In [136]:
X3_test

,team,team2,gameid,year,home_away,team_points,win_loss,disposals,kicks,inside50s,clearances,contested_possessions,contested_marks,marks_inside50,goal_assists,Model_win_loss
2410,Geelong,Melbourne,2018EF01,2018,0,-14.280614,0,1.191077,-0.466407,-1.946730,3.866869,0.652720,-18.454441,-7.426450,1.638673,0.0
2411,Melbourne,Geelong,2018EF01,2018,1,14.280614,1,-1.191077,0.466407,1.946730,-3.866869,-0.652720,18.454441,7.426450,-1.638673,1.0
2412,Sydney,Greater Western Sydney,2018EF02,2018,1,19.990005,0,0.963159,-0.796981,-4.191198,-1.369675,2.646147,10.165184,-5.879471,-9.750152,1.0
2413,Greater Western Sydney,Sydney,2018EF02,2018,0,-19.990005,1,-0.963159,0.796981,4.191198,1.369675,-2.646147,-10.165184,5.879471,9.750152,0.0
2414,Collingwood,West Coast,2018GF01,2018,0,0.000000,0,4.364065,-1.475157,1.811430,0.817327,2.053849,-10.933229,6.419074,-4.542873,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3989,Hawthorn,North Melbourne,2022R107,2022,1,55.524708,0,5.452548,3.339440,12.286834,10.168055,9.975265,25.477707,-3.919647,8.636644,1.0
3990,Fremantle,Adelaide,2022R108,2022,0,0.000000,0,-2.954251,-3.746867,-2.783428,1.928109,-6.140126,-22.679546,-13.268465,-7.930214,0.0
3991,Adelaide,Fremantle,2022R108,2022,1,0.000000,0,2.954251,3.746867,2.783428,-1.928109,6.140126,22.679546,13.268465,7.930214,1.0
3992,Gold Coast,West Coast,2022R109,2022,0,0.000000,0,1.192190,-0.093631,-10.358313,-9.998611,-3.750848,-26.517083,-20.397756,-24.570025,0.0


In [137]:
X3_test.loc[X2_test['Model_win_loss']== 0, 'Model_winning_team'] = X3_test['team2']
X3_test.loc[X2_test['Model_win_loss']== 1, 'Model_winning_team'] = X3_test['team2']
X3_test.loc[X2_test['win_loss']==1, 'Actual_winning_team'] = X3_test['team']
X3_test.loc[X2_test['win_loss']==0, 'Actual_winning_team'] = X3_test['team2']
X3_test.loc[X2_test['year']==2022, 'Actual_winning_team'] = 'Not Played Yet'

C:\Users\sampo\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\sampo\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\sampo\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [138]:
X3_test

,team,team2,gameid,year,home_away,team_points,win_loss,disposals,kicks,inside50s,clearances,contested_possessions,contested_marks,marks_inside50,goal_assists,Model_win_loss,Model_winning_team,Actual_winning_team
2410,Geelong,Melbourne,2018EF01,2018,0,-14.280614,0,1.191077,-0.466407,-1.946730,3.866869,0.652720,-18.454441,-7.426450,1.638673,0.0,Melbourne,Melbourne
2411,Melbourne,Geelong,2018EF01,2018,1,14.280614,1,-1.191077,0.466407,1.946730,-3.866869,-0.652720,18.454441,7.426450,-1.638673,1.0,Geelong,Melbourne
2412,Sydney,Greater Western Sydney,2018EF02,2018,1,19.990005,0,0.963159,-0.796981,-4.191198,-1.369675,2.646147,10.165184,-5.879471,-9.750152,1.0,Greater Western Sydney,Greater Western Sydney
2413,Greater Western Sydney,Sydney,2018EF02,2018,0,-19.990005,1,-0.963159,0.796981,4.191198,1.369675,-2.646147,-10.165184,5.879471,9.750152,0.0,Sydney,Greater Western Sydney
2414,Collingwood,West Coast,2018GF01,2018,0,0.000000,0,4.364065,-1.475157,1.811430,0.817327,2.053849,-10.933229,6.419074,-4.542873,0.0,West Coast,West Coast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3989,Hawthorn,North Melbourne,2022R107,2022,1,55.524708,0,5.452548,3.339440,12.286834,10.168055,9.975265,25.477707,-3.919647,8.636644,1.0,North Melbourne,Not Played Yet
3990,Fremantle,Adelaide,2022R108,2022,0,0.000000,0,-2.954251,-3.746867,-2.783428,1.928109,-6.140126,-22.679546,-13.268465,-7.930214,0.0,Adelaide,Not Played Yet
3991,Adelaide,Fremantle,2022R108,2022,1,0.000000,0,2.954251,3.746867,2.783428,-1.928109,6.140126,22.679546,13.268465,7.930214,1.0,Fremantle,Not Played Yet
3992,Gold Coast,West Coast,2022R109,2022,0,0.000000,0,1.192190,-0.093631,-10.358313,-9.998611,-3.750848,-26.517083,-20.397756,-24.570025,0.0,West Coast,Not Played Yet


In [139]:
# H2O Model
# H2O's AutoML function automatically creates many models, tunes each model 
# and returns the best model with the highest/lowest metric
import h2o
from h2o.automl import H2OAutoML

In [140]:
# Initialise h2o session
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 27 mins
H2O_cluster_timezone:,Australia/Sydney
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_sampo_m456bi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.209 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [141]:
train_h2o = h2o.H2OFrame(X2_train)
test_h2o = h2o.H2OFrame(X2_test)

train_h2o['win_loss'] = train_h2o['win_loss'].asfactor()
test_h2o['win_loss'] = test_h2o['win_loss'].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [142]:
train_h2o.head()

team,team2,gameid,year,home_away,team_points,win_loss,disposals,kicks,inside50s,clearances,contested_possessions,contested_marks,marks_inside50,goal_assists
Fremantle,Geelong,2012EF01,2012,0,-14.2806,1,0.291371,4.55625,-2.74198,0.27697,0.567356,-9.34915,-8.82029,-7.21278
Geelong,Fremantle,2012EF01,2012,1,14.2806,0,-0.291371,-4.55625,2.74198,-0.27697,-0.567356,9.34915,8.82029,7.21278
West Coast,North Melbourne,2012EF02,2012,1,-14.2806,1,-5.015,-4.84484,-0.404817,7.69125,2.19933,11.994,11.106,-6.02047
North Melbourne,West Coast,2012EF02,2012,0,14.2806,0,5.015,4.84484,0.404817,-7.69125,-2.19933,-11.994,-11.106,6.02047
Hawthorn,Sydney,2012GF01,2012,1,24.9922,0,-1.06156,0.525549,10.3065,4.49119,-0.59092,-29.985,-1.75362,3.56983
Sydney,Hawthorn,2012GF01,2012,0,-24.9922,1,1.06156,-0.525549,-10.3065,-4.49119,0.59092,29.985,1.75362,-3.56983
Collingwood,Sydney,2012PF01,2012,0,0,0,-3.98225,2.93832,-2.65821,-4.50869,-6.05834,6.66389,14.2797,-24.1241
Sydney,Collingwood,2012PF01,2012,1,0,1,3.98225,-2.93832,2.65821,4.50869,6.05834,-6.66389,-14.2797,24.1241
Hawthorn,Adelaide,2012PF02,2012,1,24.9922,1,4.29983,-1.20703,6.10767,12.0588,2.22524,-28.4061,-3.93546,20.3609
Adelaide,Hawthorn,2012PF02,2012,0,-24.9922,0,-4.29983,1.20703,-6.10767,-12.0588,-2.22524,28.4061,3.93546,-20.3609


In [143]:
# Create an AutoML object
aml = H2OAutoML(max_runtime_secs=60, seed=42)

# Train the model
aml.train(y='win_loss', 
          x=['disposals', 'kicks', 'inside50s','contested_possessions', 'marks_inside50', 'goal_assists', 'team_points'], 
          training_frame=train_h2o)

AutoML progress: |█
13:50:57.351: AutoML: XGBoost is not available; skipping it.
13:50:57.351: Step 'best_of_family_xgboost' not defined in provider 'StackedEnsemble': skipping it.
13:50:57.351: Step 'all_xgboost' not defined in provider 'StackedEnsemble': skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_1_AutoML_2_20220122_135057


GLM Model: summary


,,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,Ridge ( lambda = 0.008479 ),"nlambda = 30, lambda.max = 17.326, lambda.min = 0.008479, lambda.1...",7,7,32,AutoML_2_20220122_135057_training_py_7_sid_8bfb




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.21382292175301246
RMSE: 0.4624099066337274
LogLoss: 0.6154958654001264
Null degrees of freedom: 2409
Residual degrees of freedom: 2402
Null deviance: 3340.8631854520563
Residual deviance: 2966.690071228609
AIC: 2982.690071228609
AUC: 0.7174717502742842
AUCPR: 0.707067192026871
Gini: 0.43494350054856845

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3207765018065137: 


,,0,1,Error,Rate
0,0,362.0,851.0,0.7016,(851.0/1213.0)
1,1,103.0,1094.0,0.086,(103.0/1197.0)
2,Total,465.0,1945.0,0.3959,(954.0/2410.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.629461,0.043780,0.663900,0.558091,0.665975,0.624481,0.634855
1,auc,0.715172,0.020233,0.729830,0.687075,0.703690,0.717867,0.737401
2,err,0.370539,0.043780,0.336100,0.441909,0.334025,0.375519,0.365145
3,err_count,178.600000,21.102133,162.000000,213.000000,161.000000,181.000000,176.000000
4,f0point5,0.629056,0.032975,0.650545,0.574378,0.658888,0.629649,0.631821
5,f1,0.703106,0.019969,0.702206,0.670788,0.704587,0.723664,0.714286
6,f2,0.799632,0.039666,0.762780,0.806092,0.757098,0.850682,0.821509
7,lift_top_group,1.772383,0.224361,1.633898,2.068670,1.600000,1.959350,1.600000
8,logloss,0.616755,0.014192,0.609620,0.636169,0.626167,0.611107,0.600713
9,max_per_class_error,0.620694,0.147164,0.475610,0.791165,0.464730,0.728814,0.643154



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_xval,deviance_se,alpha,iterations,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-01-22 13:50:57,0.000 sec,2,.17e2,8.0,1.372166,1.375028,0.000335,0.0,NaN,,,,,,,
1,,2022-01-22 13:50:57,0.001 sec,4,.11e2,8.0,1.364629,1.368763,0.00053,0.0,NaN,,,,,,,
2,,2022-01-22 13:50:57,0.002 sec,6,.67e1,8.0,1.353886,1.359639,0.000839,0.0,NaN,,,,,,,
3,,2022-01-22 13:50:57,0.003 sec,8,.41e1,8.0,1.339507,1.347028,0.001304,0.0,NaN,,,,,,,
4,,2022-01-22 13:50:57,0.004 sec,10,.26e1,8.0,1.321828,1.330869,0.001969,0.0,NaN,,,,,,,
5,,2022-01-22 13:50:57,0.005 sec,12,.16e1,8.0,1.302311,1.312062,0.002862,0.0,NaN,,,,,,,
6,,2022-01-22 13:50:57,0.006 sec,14,.99e0,8.0,1.283277,1.292643,0.003978,0.0,NaN,,,,,,,
7,,2022-01-22 13:50:57,0.007 sec,16,.62e0,8.0,1.266864,1.275017,0.005257,0.0,NaN,,,,,,,
8,,2022-01-22 13:50:57,0.008 sec,18,.38e0,8.0,1.254247,1.260845,0.006608,0.0,NaN,,,,,,,
9,,2022-01-22 13:50:57,0.009 sec,20,.24e0,8.0,1.245395,1.250648,0.007922,0.0,NaN,,,,,,,



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,inside50s,0.342433,1.000000,0.302427
1,team_points,0.331436,0.967885,0.292715
2,contested_possessions,0.198784,0.580505,0.175561
3,marks_inside50,0.168111,0.490931,0.148471
4,disposals,0.036104,0.105434,0.031886
5,kicks,0.032543,0.095035,0.028741
6,goal_assists,0.022871,0.066789,0.020199


In [144]:
# Predict the rounds for the test data set
rd_1_predictions = aml.predict(test_h2o).as_data_frame()

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [145]:
# Evaluating model
evaluation(y_test, rd_1_predictions['predict'])

Evaluation Metrics:
Accuracy: 0.5542929292929293
Recall: 0.9099099099099099
F1 Score: 0.6669811320754717
Precision: 0.526433358153388

Confusion Matrix:
 TN,  FP, FN, TP
[171 636  70 707]


In [146]:
# View the round 1 predictions
rd_1_predictions

,predict,p0,p1
0,1,0.563039,0.436961
1,1,0.444580,0.555420
2,1,0.500748,0.499252
3,1,0.506980,0.493020
4,1,0.434620,0.565380
...,...,...,...
1579,1,0.204407,0.795593
1580,1,0.646662,0.353338
1581,1,0.360432,0.639568
1582,0,0.696706,0.303294


In [147]:
rd_1_predictions['p1']

0       0.436961
1       0.555420
2       0.499252
3       0.493020
4       0.565380
          ...   
1579    0.795593
1580    0.353338
1581    0.639568
1582    0.303294
1583    0.690135
Name: p1, Length: 1584, dtype: float64

In [148]:
X2_test

,team,team2,gameid,year,home_away,team_points,win_loss,disposals,kicks,inside50s,clearances,contested_possessions,contested_marks,marks_inside50,goal_assists,Model_win_loss,Model_winning_team,Actual_winning_team
2410,Geelong,Melbourne,2018EF01,2018,0,-14.280614,0,1.191077,-0.466407,-1.946730,3.866869,0.652720,-18.454441,-7.426450,1.638673,0.0,Melbourne,Melbourne
2411,Melbourne,Geelong,2018EF01,2018,1,14.280614,1,-1.191077,0.466407,1.946730,-3.866869,-0.652720,18.454441,7.426450,-1.638673,1.0,Geelong,Melbourne
2412,Sydney,Greater Western Sydney,2018EF02,2018,1,19.990005,0,0.963159,-0.796981,-4.191198,-1.369675,2.646147,10.165184,-5.879471,-9.750152,1.0,Greater Western Sydney,Greater Western Sydney
2413,Greater Western Sydney,Sydney,2018EF02,2018,0,-19.990005,1,-0.963159,0.796981,4.191198,1.369675,-2.646147,-10.165184,5.879471,9.750152,0.0,Sydney,Greater Western Sydney
2414,Collingwood,West Coast,2018GF01,2018,0,0.000000,0,4.364065,-1.475157,1.811430,0.817327,2.053849,-10.933229,6.419074,-4.542873,0.0,West Coast,West Coast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3989,Hawthorn,North Melbourne,2022R107,2022,1,55.524708,0,5.452548,3.339440,12.286834,10.168055,9.975265,25.477707,-3.919647,8.636644,1.0,North Melbourne,Not Played Yet
3990,Fremantle,Adelaide,2022R108,2022,0,0.000000,0,-2.954251,-3.746867,-2.783428,1.928109,-6.140126,-22.679546,-13.268465,-7.930214,0.0,Adelaide,Not Played Yet
3991,Adelaide,Fremantle,2022R108,2022,1,0.000000,0,2.954251,3.746867,2.783428,-1.928109,6.140126,22.679546,13.268465,7.930214,1.0,Fremantle,Not Played Yet
3992,Gold Coast,West Coast,2022R109,2022,0,0.000000,0,1.192190,-0.093631,-10.358313,-9.998611,-3.750848,-26.517083,-20.397756,-24.570025,0.0,West Coast,Not Played Yet


In [149]:
# Add the round 1 predictions to the test dataframe
#X2_test['model_prob_1'] = rd_1_predictions.p1
X2_test['model_prob_1'] = rd_1_predictions['p1'].values
#X2_test['model_prob_2'] = rd_1_predictions.p0
X2_test['model_prob_2'] = rd_1_predictions['p0'].values
X2_test['model_odds_1'] = 1 / X2_test['model_prob_1'].values
X2_test['model_odds_2'] = 1 / X2_test['model_prob_2'].values

<ipython-input-149-11207e084977>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X2_test['model_prob_1'] = rd_1_predictions['p1'].values
<ipython-input-149-11207e084977>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X2_test['model_prob_2'] = rd_1_predictions['p0'].values
<ipython-input-149-11207e084977>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [150]:
# add the predictions to the data set and provide the model/actual winning team names

X2_test.loc[X2_test['model_prob_1']>= X2_test['model_prob_2'], 'Model_winning_team'] = X2_test['team']
X2_test.loc[X2_test['model_prob_1']< X2_test['model_prob_2'], 'Model_winning_team'] = X2_test['team2']
X2_test.loc[X2_test['win_loss']==1, 'Actual_winning_team'] = X2_test['team']
X2_test.loc[X2_test['win_loss']==0, 'Actual_winning_team'] = X2_test['team2']
X2_test.loc[X2_test['year']==2022, 'Actual_winning_team'] = 'Not Played Yet'

C:\Users\sampo\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\sampo\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\sampo\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [151]:
X2_test

,team,team2,gameid,year,home_away,team_points,win_loss,disposals,kicks,inside50s,clearances,contested_possessions,contested_marks,marks_inside50,goal_assists,Model_win_loss,Model_winning_team,Actual_winning_team,model_prob_1,model_prob_2,model_odds_1,model_odds_2
2410,Geelong,Melbourne,2018EF01,2018,0,-14.280614,0,1.191077,-0.466407,-1.946730,3.866869,0.652720,-18.454441,-7.426450,1.638673,0.0,Melbourne,Melbourne,0.436961,0.563039,2.288533,1.776076
2411,Melbourne,Geelong,2018EF01,2018,1,14.280614,1,-1.191077,0.466407,1.946730,-3.866869,-0.652720,18.454441,7.426450,-1.638673,1.0,Melbourne,Melbourne,0.555420,0.444580,1.800439,2.249314
2412,Sydney,Greater Western Sydney,2018EF02,2018,1,19.990005,0,0.963159,-0.796981,-4.191198,-1.369675,2.646147,10.165184,-5.879471,-9.750152,1.0,Greater Western Sydney,Greater Western Sydney,0.499252,0.500748,2.002995,1.997014
2413,Greater Western Sydney,Sydney,2018EF02,2018,0,-19.990005,1,-0.963159,0.796981,4.191198,1.369675,-2.646147,-10.165184,5.879471,9.750152,0.0,Sydney,Greater Western Sydney,0.493020,0.506980,2.028313,1.972466
2414,Collingwood,West Coast,2018GF01,2018,0,0.000000,0,4.364065,-1.475157,1.811430,0.817327,2.053849,-10.933229,6.419074,-4.542873,0.0,Collingwood,West Coast,0.565380,0.434620,1.768722,2.300860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3989,Hawthorn,North Melbourne,2022R107,2022,1,55.524708,0,5.452548,3.339440,12.286834,10.168055,9.975265,25.477707,-3.919647,8.636644,1.0,Hawthorn,Not Played Yet,0.795593,0.204407,1.256924,4.892201
3990,Fremantle,Adelaide,2022R108,2022,0,0.000000,0,-2.954251,-3.746867,-2.783428,1.928109,-6.140126,-22.679546,-13.268465,-7.930214,0.0,Adelaide,Not Played Yet,0.353338,0.646662,2.830151,1.546403
3991,Adelaide,Fremantle,2022R108,2022,1,0.000000,0,2.954251,3.746867,2.783428,-1.928109,6.140126,22.679546,13.268465,7.930214,1.0,Adelaide,Not Played Yet,0.639568,0.360432,1.563556,2.774446
3992,Gold Coast,West Coast,2022R109,2022,0,0.000000,0,1.192190,-0.093631,-10.358313,-9.998611,-3.750848,-26.517083,-20.397756,-24.570025,0.0,West Coast,Not Played Yet,0.303294,0.696706,3.297127,1.435326


In [152]:
# View the predictions
X2_test[['gameid','team', 'team2', 'model_prob_1','model_prob_2', 'win_loss', 'Actual_winning_team', 'Model_winning_team']].head(50)

,gameid,team,team2,model_prob_1,model_prob_2,win_loss,Actual_winning_team,Model_winning_team
2410,2018EF01,Geelong,Melbourne,0.436961,0.563039,0,Melbourne,Melbourne
2411,2018EF01,Melbourne,Geelong,0.555420,0.444580,1,Melbourne,Melbourne
2412,2018EF02,Sydney,Greater Western Sydney,0.499252,0.500748,0,Greater Western Sydney,Greater Western Sydney
2413,2018EF02,Greater Western Sydney,Sydney,0.493020,0.506980,1,Greater Western Sydney,Sydney
2414,2018GF01,Collingwood,West Coast,0.565380,0.434620,0,West Coast,Collingwood
2415,2018GF01,West Coast,Collingwood,0.427041,0.572959,1,West Coast,Collingwood
2416,2018PF01,Richmond,Collingwood,0.560753,0.439247,0,Collingwood,Richmond
2417,2018PF01,Collingwood,Richmond,0.431648,0.568352,1,Collingwood,Richmond
2418,2018PF02,West Coast,Melbourne,0.431359,0.568641,1,West Coast,Melbourne
2419,2018PF02,Melbourne,West Coast,0.561044,0.438956,0,West Coast,Melbourne


In [153]:
# select columns to combine with 5 game team statistics
df1 = X2_test[['gameid','team', 'model_prob_1','model_prob_2', 'model_odds_1','model_odds_2','Actual_winning_team', 'Model_winning_team']]

In [154]:
# select columns to combine with 5 game team statistics
df3 = X3_test[['gameid','team', 'model_prob_1','Actual_winning_team', 'Model_winning_team']]

In [155]:
# read in 5 game team statistics
df2 = pd.read_sql_query('select * from "afl_team_performance_last_5_games"',con=engine)

In [156]:
# merge the datasets on the basis of team and gameid
merged_df = df1.merge(df2, how='inner', left_on=["team", "gameid"], right_on=["team", "gameid"])
merged_df3 = df3.merge(df2, how='inner', left_on=["team", "gameid"], right_on=["team", "gameid"])

In [157]:
merged_df

,gameid,team,model_prob_1,model_prob_2,model_odds_1,model_odds_2,Actual_winning_team,Model_winning_team,team2,date,year,round,venue,starttime,win_loss,home_away,team_score,rainfall,team_points,opposing_team_score,win_loss_margin,win_loss_margin_percent,disposals,kicks,marks,handballs,goals,behinds,hitouts,tackles,rebounds,inside50s,clearances,clangers,frees,frees_against,contested_possessions,uncontested_possessions,contested_marks,marks_inside50,one_percenters,bounces,goal_assists
0,2018EF01,Geelong,0.436961,0.563039,2.288533,1.776076,Melbourne,Melbourne,Melbourne,2018-09-07,2018,EF,M.C.G.,19:50:00,0,0,112.4,2.32,12,59.8,52.6,163.8,390.8,213.4,90.4,177.4,16.8,10.6,34.2,68.8,36.4,55.4,37.6,52.0,22.6,19.2,154.2,244.8,10.6,16.2,54.2,1.8,12.4
1,2018EF01,Melbourne,0.555420,0.444580,1.800439,2.249314,Melbourne,Melbourne,Geelong,2018-09-07,2018,EF,M.C.G.,19:50:00,1,1,107.0,1.40,16,70.6,36.4,67.2,381.6,215.4,98.4,166.2,15.6,10.6,46.4,68.2,34.6,57.6,34.8,52.6,18.4,17.8,152.2,233.6,15.4,18.8,43.8,5.8,12.0
2,2018EF02,Sydney,0.499252,0.500748,2.002995,1.997014,Greater Western Sydney,Greater Western Sydney,Greater Western Sydney,2018-09-08,2018,EF,S.C.G.,16:20:00,0,1,76.0,1.52,12,80.2,-4.2,-5.8,356.4,211.6,96.4,144.8,11.0,9.2,28.0,71.6,42.4,48.0,36.0,53.4,21.4,20.2,147.4,209.8,13.0,9.6,45.4,3.4,7.4
3,2018EF02,Greater Western Sydney,0.493020,0.506980,2.028313,1.972466,Greater Western Sydney,Sydney,Sydney,2018-09-08,2018,EF,S.C.G.,16:20:00,1,0,86.6,0.16,8,77.8,8.8,22.8,349.6,215.0,90.4,134.6,12.6,8.2,34.8,68.4,37.8,52.2,37.0,51.6,16.2,20.2,139.8,210.6,10.6,10.8,50.4,6.6,9.0
4,2018GF01,Collingwood,0.565380,0.434620,1.768722,2.300860,West Coast,Collingwood,West Coast,2018-09-29,2018,GF,M.C.G.,14:30:00,0,0,85.4,0.72,16,66.8,18.6,30.8,389.8,220.4,84.2,169.4,12.4,8.6,49.2,72.4,33.8,56.2,37.0,57.4,19.4,16.8,159.0,233.6,11.4,11.6,60.0,2.4,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,2022R107,Hawthorn,0.795593,0.204407,1.256924,4.892201,Not Played Yet,Hawthorn,North Melbourne,2022-03-20,2022,R1,MCG,13:10:00,0,1,83.8,3.08,14,76.0,7.8,17.8,398.4,222.8,105.2,175.6,12.6,6.0,45.0,61.0,32.4,53.0,39.0,49.6,16.6,16.2,135.6,259.4,12.8,9.8,47.0,5.6,8.8
1580,2022R108,Fremantle,0.353338,0.646662,2.830151,1.546403,Not Played Yet,Adelaide,Adelaide,2022-03-20,2022,R1,Adelaide Oval,16:10:00,0,0,59.0,2.52,8,87.6,-28.6,-55.0,348.2,197.8,86.4,150.4,8.2,7.8,37.8,48.4,39.4,45.4,37.0,55.0,17.8,17.0,128.4,217.6,9.2,9.8,38.4,5.6,5.8
1581,2022R108,Adelaide,0.639568,0.360432,1.563556,2.774446,Not Played Yet,Adelaide,Fremantle,2022-03-20,2022,R1,Adelaide Oval,16:10:00,0,1,74.0,0.32,8,80.2,-6.2,-11.4,369.4,213.2,85.2,156.2,10.6,8.2,34.8,58.4,41.2,48.0,35.6,60.2,20.0,21.6,145.2,218.6,14.6,12.8,43.0,4.8,6.8
1582,2022R109,Gold Coast,0.303294,0.696706,3.297127,1.435326,Not Played Yet,West Coast,West Coast,2022-03-20,2022,R1,Optus Stadium,19:40:00,0,0,51.2,5.08,4,107.8,-56.6,-153.6,348.0,213.4,100.6,134.6,7.0,7.0,27.2,53.6,45.0,42.4,32.4,63.2,18.0,20.4,120.6,222.6,7.2,7.8,51.8,6.0,4.6


In [158]:
# save dataframe to json & csv file
merged_df.to_json('Resources/model_export.json')
merged_df.to_csv('Resources/model_export.csv', index=False)
merged_df3.to_csv('Resources/model_export_lr.csv', index=False)

In [2]:
import pandas as pd


In [4]:
df = pd.read_json('./Resources/model_export.json')
print(df)

        gameid                    team  model_prob_1  model_prob_2  \
0     2018EF01                 Geelong      0.436961      0.563039   
1     2018EF01               Melbourne      0.555420      0.444580   
2     2018EF02                  Sydney      0.499252      0.500748   
3     2018EF02  Greater Western Sydney      0.493020      0.506980   
4     2018GF01             Collingwood      0.565380      0.434620   
...        ...                     ...           ...           ...   
1579  2022R107                Hawthorn      0.795593      0.204407   
1580  2022R108               Fremantle      0.353338      0.646662   
1581  2022R108                Adelaide      0.639568      0.360432   
1582  2022R109              Gold Coast      0.303294      0.696706   
1583  2022R109              West Coast      0.690135      0.309865   

      model_odds_1  model_odds_2     Actual_winning_team  \
0         2.288533      1.776076               Melbourne   
1         1.800439      2.249314       

In [5]:
data = df.to_json(orient='columns')

In [6]:
print(data)

{"gameid":{"0":"2018EF01","1":"2018EF01","2":"2018EF02","3":"2018EF02","4":"2018GF01","5":"2018GF01","6":"2018PF01","7":"2018PF01","8":"2018PF02","9":"2018PF02","10":"2018QF01","11":"2018QF01","12":"2018QF02","13":"2018QF02","14":"2018R1001","15":"2018R1001","16":"2018R1002","17":"2018R1002","18":"2018R1003","19":"2018R1003","20":"2018R1004","21":"2018R1004","22":"2018R1005","23":"2018R1005","24":"2018R1006","25":"2018R1006","26":"2018R1007","27":"2018R1007","28":"2018R1008","29":"2018R1008","30":"2018R101","31":"2018R101","32":"2018R102","33":"2018R102","34":"2018R103","35":"2018R103","36":"2018R104","37":"2018R104","38":"2018R105","39":"2018R105","40":"2018R106","41":"2018R106","42":"2018R107","43":"2018R107","44":"2018R108","45":"2018R108","46":"2018R109","47":"2018R109","48":"2018R1101","49":"2018R1101","50":"2018R1102","51":"2018R1102","52":"2018R1103","53":"2018R1103","54":"2018R1104","55":"2018R1104","56":"2018R1105","57":"2018R1105","58":"2018R1106","59":"2018R1106","60":"2018R

In [7]:
New_df = df.to_dict('index')

In [8]:
print(New_df)

{0: {'gameid': '2018EF01', 'team': 'Geelong', 'model_prob_1': 0.43696108920000004, 'model_prob_2': 0.5630389108, 'model_odds_1': 2.2885332921, 'model_odds_2': 1.7760761837999999, 'Actual_winning_team': 'Melbourne', 'Model_winning_team': 'Melbourne', 'team2': 'Melbourne', 'date': Timestamp('2018-09-07 00:00:00'), 'year': 2018, 'round': 'EF', 'venue': 'M.C.G.', 'starttime': '19:50:00', 'win_loss': 0, 'home_away': 0, 'team_score': 112.4, 'rainfall': 2.32, 'team_points': 12, 'opposing_team_score': 59.8, 'win_loss_margin': 52.6, 'win_loss_margin_percent': 163.8, 'disposals': 390.8, 'kicks': 213.4, 'marks': 90.4, 'handballs': 177.4, 'goals': 16.8, 'behinds': 10.6, 'hitouts': 34.2, 'tackles': 68.8, 'rebounds': 36.4, 'inside50s': 55.4, 'clearances': 37.6, 'clangers': 52.0, 'frees': 22.6, 'frees_against': 19.2, 'contested_possessions': 154.2, 'uncontested_possessions': 244.8, 'contested_marks': 10.6, 'marks_inside50': 16.2, 'one_percenters': 54.2, 'bounces': 1.8, 'goal_assists': 12.4}, 1: {'gam